In [55]:
!pip install -q "numpy<2.0" opencv-python-headless ultralytics huggingface_hub matplotlib scipy roboflow supervision

In [56]:
import cv2
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
import os
from collections import deque
from PIL import Image

import numpy as np, math
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment

from typing import Dict, Any
from kaggle_secrets import UserSecretsClient
from roboflow import Roboflow

In [57]:
user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("ROBOFLOW_API_KEY")

In [58]:
# --- STEP 1: LOAD "LITTLE YOLO" ---
# We use 'yolov8n.pt' (n = nano). It is the smallest, fastest model.
# It automatically downloads from Hugging Face / Ultralytics if not found.
model = YOLO('yolov8n.pt')
print("Model loaded successfully.")

Model loaded successfully.


In [59]:
def save_clean_chess_frames(video_path, output_folder="clean_frame", save_step=True):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    # --- SETTINGS ---
    FRAME_SKIP = 15       # Look at 1 frame, skip 14
    CONFIDENCE = 0.125    # Be 40% sure it's a hand
    BUFFER_SIZE = 2       # Must see same state 2 times to switch
    
    # --- STATE VARIABLES ---
    frame_count = 0
    chunk_counter = 1     # We will number our chunks: 1, 2, 3...
    
    current_official_state = None  # The state we are currently recording
    
    # A small memory buffer to store recent results
    result_buffer = deque(maxlen=BUFFER_SIZE)  # (frame_image, frame_number, state_string)

    print(f"Processing... Saving ordered chunks to '/{output_folder}'")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            continue  # Skip frames for speed

        # 1. DETECT HAND / BLOCK
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)

        results_list = model(
            [pil_img],          
            verbose=False,
            conf=CONFIDENCE,
            classes=[0],       # class 0 = hand (in your small YOLO)
        )

        result = results_list[0]
        is_blocked = len(result.boxes) > 0
        frame_state = "blocked" if is_blocked else "clean"
        
        # 2. ADD TO BUFFER
        result_buffer.append((frame.copy(), frame_count, frame_state))
        
        if len(result_buffer) < BUFFER_SIZE:
            continue

        # 3. CHECK FOR CONSENSUS
        recent_states = [x[2] for x in result_buffer]
        if not all(s == recent_states[0] for s in recent_states):
            continue

        detected_stable_state = recent_states[0]
            
        # --- SCENARIO A: FIRST CHUNK EVER ---
        if current_official_state is None:
            current_official_state = detected_stable_state
            img, num, state = result_buffer[0]
            filename = f"{output_folder}/{chunk_counter:02d}_frame{num}_START_{state}.jpg"
            cv2.imwrite(filename, img)
            print(f"[{chunk_counter:03d}] Started {state} at frame {num}")

        # --- SCENARIO B: STATE SWITCH ---
        elif current_official_state != detected_stable_state:
            # 1. Close OLD chunk
            if save_step:
                img_end, num_end, _ = result_buffer[0]
                filename_end = f"{output_folder}/{chunk_counter:02d}_frame{num_end}_END_{current_official_state}.jpg"
                cv2.imwrite(filename_end, img_end)
            
            # 2. Start NEW chunk
            chunk_counter += 1
            current_official_state = detected_stable_state
            
            img_start, num_start, state_start = result_buffer[-1]
            if state_start == "clean" or save_step:
                filename_start = f"{output_folder}/{chunk_counter:02d}_frame{num_start}_START_{state_start}.jpg"
                cv2.imwrite(filename_start, img_start)
            
            print(f"[{chunk_counter:03d}] Switched to {state_start} at frame {num_start}")

    # --- FINAL CLEANUP ---
    if len(result_buffer) > 0 and current_official_state is not None and save_step:
        img_last, num_last, _ = result_buffer[-1]
        filename_final = f"{output_folder}/{chunk_counter:02d}_frame{num_last}_END_{current_official_state}.jpg"
        cv2.imwrite(filename_final, img_last)

    cap.release()
    print("Done.")

    

In [60]:
# --- RUN IT ---
video_file = "/kaggle/input/cu-chess-detection-2025/Chess Detection Competition/test_videos/2_move_student.mp4"
save_clean_chess_frames(video_file, output_folder="2_Move_student", save_step=False)


Processing... Saving ordered chunks to '/2_Move_student'
[001] Started clean at frame 15
[002] Switched to blocked at frame 75
[003] Switched to clean at frame 420
[004] Switched to blocked at frame 1155
[005] Switched to clean at frame 1230
Done.


## 8 by 8 grid

In [61]:
def crop_board(bgr_img): 
    """ TEMP: later this will crop the chessboard using YOLO or corner detection. 
    For now, just return the original image unchanged. """ 
    return bgr_img.copy()

In [62]:
def rotate(bgr_img):
    """
    TEMP: Rotate image so that the WHITE side is at the bottom.
    """
    return bgr_img.copy()


In [63]:
MAX_W = 640

# Preprocess
GAUSS_K = (5,5)
CANNY_LOW, CANNY_HIGH = 70, 170
DILATE_K, ERODE_K = (5,5), (5,5)

# Long-line anchors (for clustering only)
MIN_LEN_RATIO = 0.05

# Join/merge params (scale with image diagonal)
ANGLE_TOL_DEG      = 10.0
RHO_EPS_RATIO      = 0.020
GAP_EPS_RATIO      = 0.010
POST_MIN_LEN_RATIO = 0.15

# Inner mask (optional)
MASK_ERODE_RATIO_INNER = 0.07

# Texture heuristic
XSEC_HALF_PIX   = 6
XSEC_NSAMPLES   = 12

# Quality mix
ANG_NORM_DEG       = 5.0
BETA_ANG           = 0.70
BETA_LEN0          = 0.15
BETA_LEN1          = 0.85
TEXT_BONUS         = 0.9

# Selection weights & guards
W_RHO = 1.2        
W_Q   = 0.25       
INNER_BAND_MARGIN = 0.10  
FAR_GATE_MULT     = 2.5   
MIN_SEP_MULT      = 0.39 
LOW_SEP_MULT      = 0.50  
HIGH_SEP_MULT     = 1.50  
MIN_COVER         = 0.50  

# EXACT number of lines per family
N_GRID = 9

In [64]:
# ---------------- helpers ----------------

def preprocess_edges(bgr):
    g = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(g, GAUSS_K, 0)
    e = cv2.Canny(g, CANNY_LOW, CANNY_HIGH)
    e = cv2.dilate(e, cv2.getStructuringElement(cv2.MORPH_RECT, DILATE_K), 1)
    e = cv2.erode( e, cv2.getStructuringElement(cv2.MORPH_RECT, ERODE_K), 1)
    return e

def erode_mask_by_ratio(mask, ratio):
    h,w = mask.shape[:2]
    k = max(3, int(ratio * min(h,w)))
    if k % 2 == 0: k += 1
    return cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k,k)), 1)

def lsd_segments(gray, mask=None, min_len_ratio=0.0):
    h, w = gray.shape[:2]
    diag = math.hypot(w, h)
    min_len = max(0.0, float(min_len_ratio) * diag)
    lsd = cv2.createLineSegmentDetector(cv2.LSD_REFINE_ADV)
    g = gray if mask is None else cv2.bitwise_and(gray, gray, mask=mask)
    lines = lsd.detect(g)[0]
    if lines is None:
        return np.zeros((0,4), np.float32)
    segs = lines[:,0,:].astype(np.float32)
    L = np.hypot(segs[:,2]-segs[:,0], segs[:,3]-segs[:,1])
    segs = segs[L >= min_len]
    vis = cv2.cvtColor(g, cv2.COLOR_GRAY2BGR)
    for x1,y1,x2,y2 in segs.astype(int):
        cv2.line(vis, (x1,y1), (x2,y2), (0,255,0), 2, cv2.LINE_AA)
    return segs

def seg_angle(seg):
    x1,y1,x2,y2 = seg
    a = math.atan2(y2-y1, x2-x1)
    if a < 0: a += math.pi
    return a

def cluster_two_families(lines):
    if len(lines) < 2: return None, None
    ang = np.array([seg_angle(s) for s in lines], np.float32)
    feats = np.c_[np.cos(2*ang), np.sin(2*ang)].astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-3)
    _, lab, _ = cv2.kmeans(feats, 2, None, crit, 10, cv2.KMEANS_PP_CENTERS)
    lab = lab.ravel()
    return lines[lab==0], lines[lab==1]

def mean_angle_of_family(family):
    if len(family) == 0: return None
    ang = np.array([seg_angle(s) for s in family], np.float64)
    c = np.cos(2*ang).mean(); s = np.sin(2*ang).mean()
    a2 = math.atan2(s, c);  a2 = a2 if a2 >= 0 else a2 + 2*math.pi
    return 0.5 * a2

def angle_diff(a, b):
    d = abs(a - b);  return min(d, math.pi - d)

def draw_segments(img, segs, color=(255,0,0), thick=3):
    vis = img.copy()
    for x1,y1,x2,y2 in segs.astype(int):
        cv2.line(vis, (x1,y1), (x2,y2), color, thick, cv2.LINE_AA)
    return vis

# ---------- merging (preserve per-line angles) ----------
def _pca_dir_from_segments(segs):
    dirs=[]
    for s in segs:
        dx,dy = s[2]-s[0], s[3]-s[1]
        n = math.hypot(dx,dy) + 1e-12
        dirs.append([dx/n, dy/n])
    D = np.array(dirs, np.float64)
    if len(D) == 1: u = D[0]
    else:
        U,S,Vt = np.linalg.svd(D - D.mean(0))
        u = Vt[0]
    u /= (np.linalg.norm(u)+1e-12)
    return u

def _project_interval(seg, u, n):
    p1 = np.array([seg[0], seg[1]], dtype=np.float64)
    p2 = np.array([seg[2], seg[3]], dtype=np.float64)
    t1 = float(np.dot(u, p1)); t2 = float(np.dot(u, p2))
    t_lo, t_hi = (t1, t2) if t1 <= t2 else (t2, t1)
    mid = 0.5*(p1+p2)
    rho = float(np.dot(n, mid))
    return t_lo, t_hi, rho

def merge_family_preserve_angles(segs, theta_family, diag,
                                 rho_eps_ratio=RHO_EPS_RATIO, gap_eps_ratio=GAP_EPS_RATIO):
    if len(segs) == 0: return segs
    uF = np.array([math.cos(theta_family), math.sin(theta_family)], np.float64)
    nF = np.array([-uF[1], uF[0]], np.float64)
    rho_eps = rho_eps_ratio * diag
    gap_eps = gap_eps_ratio * diag
    pad = 0.02 * diag

    mids = 0.5*(segs[:,0:2] + segs[:,2:4])
    rhoF = mids @ nF
    order = np.argsort(rhoF)
    segs = segs[order]; rhoF = rhoF[order]

    merged = []
    i = 0; N = len(segs)
    while i < N:
        j = i+1
        while j < N and abs(rhoF[j]-rhoF[i]) <= rho_eps:
            j += 1
        bin_segs = segs[i:j]

        u = _pca_dir_from_segments(bin_segs)
        n = np.array([-u[1], u[0]], np.float64)

        items = []
        for s in bin_segs:
            t_lo, t_hi, rho = _project_interval(s, u, n)
            items.append([t_lo - pad, t_hi + pad, rho])
        items = np.array(items, np.float64)

        rho_mean = float(items[:,2].mean())
        intervals = items[:,0:2]
        intervals = intervals[np.argsort(intervals[:,0])]

        cur_lo, cur_hi = intervals[0,0], intervals[0,1]
        outs = []
        for k in range(1, len(intervals)):
            a,b = intervals[k]
            if a <= cur_hi + gap_eps: cur_hi = max(cur_hi, b)
            else: outs.append((cur_lo, cur_hi)); cur_lo, cur_hi = a, b
        outs.append((cur_lo, cur_hi))

        for (a,b) in outs:
            p1 = u*a + n*rho_mean
            p2 = u*b + n*rho_mean
            merged.append([p1[0], p1[1], p2[0], p2[1]])

        i = j

    return np.array(merged, np.float32)

def nms_by_rho(segs, theta, diag, eps_ratio=0.010):
    if len(segs)==0: return segs
    u = np.array([math.cos(theta), math.sin(theta)], np.float64)
    n = np.array([-u[1], u[0]], np.float64)
    rho = (0.5*(segs[:,0:2] + segs[:,2:4])) @ n
    order = np.argsort(rho)
    segs = segs[order]; rho = rho[order]
    eps = eps_ratio * diag
    kept = []; i = 0
    while i < len(segs):
        j = i+1
        best_k, best_len = i, np.hypot(*(segs[i,2:4]-segs[i,0:2]))
        while j < len(segs) and abs(rho[j]-rho[i]) <= eps:
            L = np.hypot(*(segs[j,2:4]-segs[j,0:2]))
            if L > best_len: best_k, best_len = j, L
            j += 1
        kept.append(segs[best_k]); i = j
    return np.array(kept, np.float32)

# ----- geometry helpers -----
def family_unit(theta):
    u = np.array([math.cos(theta), math.sin(theta)], np.float64)
    n = np.array([-u[1], u[0]], np.float64)
    return u, n

def edge_angle(a,b):
    return seg_angle(np.array([a[0],a[1],b[0],b[1]], np.float32))

def quad_edges(quad):
    return [(quad[i], quad[(i+1)%4]) for i in range(4)]

def family_board_rho_limits(quad, theta):
    u, n = family_unit(theta)
    edges = quad_edges(quad)
    angs = [edge_angle(a,b) for a,b in edges]
    diffs = [angle_diff(ang, theta) for ang in angs]
    idx = np.argsort(diffs)[:2]
    rhos = [np.dot(n, 0.5*(edges[i][0]+edges[i][1])) for i in idx]
    rho_min, rho_max = (min(rhos), max(rhos))
    all_rho = [np.dot(n, p) for p in quad]
    span = max(all_rho) - min(all_rho)
    if (rho_max - rho_min) < 0.3*span:
        rho_min, rho_max = min(all_rho), max(all_rho)
    return rho_min, rho_max

# ----- texture / measures -----
def bilinear_sample(gray, x, y):
    h,w = gray.shape
    if x<0 or y<0 or x>w-1 or y>h-1: return None
    x0,y0 = int(np.floor(x)), int(np.floor(y))
    x1,y1 = min(x0+1,w-1), min(y0+1,h-1)
    a,b = x-x0, y-y0
    v = (1-a)*(1-b)*gray[y0,x0] + a*(1-b)*gray[y0,x1] + (1-a)*b*gray[y1,x0] + a*b*gray[y1,x1]
    return float(v)

def cross_section_score(gray, seg, n_samples=XSEC_NSAMPLES, half=XSEC_HALF_PIX):
    x1,y1,x2,y2 = seg
    vx, vy = x2-x1, y2-y1
    L = math.hypot(vx, vy) + 1e-9
    ux, uy = vx/L, vy/L
    nx, ny = -uy, ux
    scores = []
    for t in np.linspace(0.15, 0.85, n_samples):
        cx, cy = x1 + t*vx, y1 + t*vy
        vals = []
        for s in range(-half, half+1):
            v = bilinear_sample(gray, cx + s*nx, cy + s*ny)
            if v is not None: vals.append(v)
        if len(vals) >= 5:
            scores.append(np.std(vals))
    return float(np.mean(scores)) if scores else 0.0

# ----- line algebra & drawing -----
def seg_to_line(seg):
    x1,y1,x2,y2 = seg
    p1 = np.array([x1,y1,1.0]); p2 = np.array([x2,y2,1.0])
    l = np.cross(p1, p2)
    n = np.linalg.norm(l[:2]) + 1e-12
    return l / n

def safe_endpoints_from_line_in_image(l, w, h):
    a, b, c = float(l[0]), float(l[1]), float(l[2])
    eps = 1e-12
    n2 = a*a + b*b
    if n2 < eps:
        return (0.0, 0.0), (w-1.0, h-1.0)
    p0x = (-c) * a / n2
    p0y = (-c) * b / n2
    un = math.hypot(-b, a)
    ux, uy = (-b / max(un,1e-12), a / max(un,1e-12))

    x_min, x_max = -1.0, w + 1.0
    y_min, y_max = -1.0, h + 1.0
    cand = []
    if abs(ux) > eps:
        for X in (x_min, x_max-1.0):
            t = (X - p0x) / ux; y = p0y + t*uy
            if y_min <= y <= y_max-1.0: cand.append((X, y))
    if abs(uy) > eps:
        for Y in (y_min, y_max-1.0):
            t = (Y - p0y) / uy; x = p0x + t*ux
            if x_min <= x <= x_max-1.0: cand.append((x, Y))
    uniq = []
    for x,y in cand:
        if not any((abs(x-x2)<1e-6 and abs(y-y2)<1e-6) for (x2,y2) in uniq):
            uniq.append((x,y))
    if len(uniq) >= 2:
        best, bestd = (uniq[0], uniq[1]), -1.0
        for i in range(len(uniq)):
            for j in range(i+1,len(uniq)):
                dx = uniq[i][0]-uniq[j][0]; dy = uniq[i][1]-uniq[j][1]
                d2 = dx*dx + dy*dy
                if d2 > bestd: bestd, best = d2, (uniq[i], uniq[j])
        def clamp(p): return (max(0.0,min(w-1.0,p[0])), max(0.0,min(h-1.0,p[1])))
        return clamp(best[0]), clamp(best[1])

    t_big = max(w,h)*2.0
    p1x, p1y = p0x - t_big*ux, p0y - t_big*uy
    p2x, p2y = p0x + t_big*ux, p0y + t_big*uy
    p1x = max(0.0, min(w-1.0, p1x)); p1y = max(0.0, min(h-1.0, p1y))
    p2x = max(0.0, min(w-1.0, p2x)); p2y = max(0.0, min(h-1.0, p2y))
    return (p1x, p1y), (p2x, p2y)

def draw_full_lines(img, segs, color, thick=2):
    vis = img.copy()
    h, w = img.shape[:2]
    for s in segs:
        l = seg_to_line(s)
        (x1,y1),(x2,y2) = safe_endpoints_from_line_in_image(l, w, h)
        if (abs(x1-x2)+abs(y1-y2)) < 1.0:
            a,b,c = float(l[0]), float(l[1]), float(l[2])
            n2 = a*a + b*b
            if n2 < 1e-12: continue
            p0x = (-c) * a / n2; p0y = (-c) * b / n2
            un = math.hypot(-b, a); ux, uy = (-b/max(un,1e-12), a/max(un,1e-12))
            R = 100.0
            x1,y1 = p0x-R*ux, p0y-R*uy
            x2,y2 = p0x+R*ux, p0y+R*uy
            x1 = max(0.0, min(w-1.0, x1)); y1 = max(0.0, min(h-1.0, y1))
            x2 = max(0.0, min(w-1.0, x2)); y2 = max(0.0, min(h-1.0, y2))
        cv2.line(vis, (int(round(x1)),int(round(y1))), (int(round(x2)),int(round(y2))), color, thick, cv2.LINE_AA)
    return vis

def all_intersections(fam1_lines, fam2_lines, shape):
    h, w = shape[:2]
    L1 = [seg_to_line(s) for s in fam1_lines]
    L2 = [seg_to_line(s) for s in fam2_lines]
    pts = []
    for l1 in L1:
        for l2 in L2:
            p = np.cross(l1, l2)
            if abs(p[2]) < 1e-12: continue
            x,y = float(p[0]/p[2]), float(p[1]/p[2])
            if -2 <= x <= w+2 and -2 <= y <= h+2:
                pts.append([x,y])
    return np.array(pts, np.float32) if pts else np.zeros((0,2), np.float32)

# ---------- coverage & span utilities ----------
def line_board_coverage(seg, mask):
    if mask is None: return 1.0
    h, w = mask.shape[:2]
    x1,y1,x2,y2 = map(float, seg)
    L = max(1.0, math.hypot(x2-x1, y2-y1))
    nS = int(max(20, L/8))
    inside = 0
    for t in np.linspace(0.05, 0.95, nS):
        x = x1 + t*(x2-x1); y = y1 + t*(y2-y1)
        xi = int(round(min(max(0, x), w-1)))
        yi = int(round(min(max(0, y), h-1)))
        inside += 1 if mask[yi, xi] > 0 else 0
    return inside / nS

def rho_span_and_d(theta, quad, n):
    uF, nF = family_unit(theta)
    rmin, rmax = family_board_rho_limits(quad, theta)
    d = (rmax - rmin) / max(n-1, 1)
    return rmin, rmax, d, nF

def filter_pool_by_inner_band(pool, theta, quad, n, margin_ratio, nF, mask_inner):
    """Keep only candidates whose ρ is within the inner band AND have good coverage."""
    if len(pool) == 0:
        return pool, 0.0
    rmin, rmax = family_board_rho_limits(quad, theta)
    d = (rmax - rmin) / max(n-1, 1)
    lo = rmin + margin_ratio * d
    hi = rmax - margin_ratio * d

    mids = 0.5*(pool[:,0:2] + pool[:,2:4])
    rho  = (mids @ nF).astype(np.float64)
    cov  = np.array([line_board_coverage(s, mask_inner) for s in pool], np.float64)

    keep = (rho >= lo) & (rho <= hi) & (cov >= MIN_COVER)
    return pool[keep], d

def select_n_hungarian_from_pool(pool, theta, n, gray, quad, mask_inner):
    """
    Pick ~n lines from 'pool' by assigning them to n evenly-spaced centers
    inside an inner band; fall back by relaxing MIN_SEP_MULT if we get < n.
    """
    uF, nF = family_unit(theta)

    # board span
    rmin, rmax = family_board_rho_limits(quad, theta)
    d = (rmax - rmin) / max(n-1, 1)

    # Use inner-band centers (prevents picking extreme rim)
    lo = rmin + INNER_BAND_MARGIN * d
    hi = rmax - INNER_BAND_MARGIN * d
    d_in = (hi - lo) / max(n-1, 1)
    centers = lo + d_in * np.arange(n, dtype=np.float64)

    # measures
    if len(pool) == 0:
        return pool

    mids = 0.5*(pool[:,0:2] + pool[:,2:4])
    rho  = (mids @ nF).astype(np.float64)
    L    = np.hypot(pool[:,2]-pool[:,0], pool[:,3]-pool[:,1]).astype(np.float64)
    Adev = np.array([angle_diff(seg_angle(s), theta) for s in pool], np.float64)
    tex  = np.array([cross_section_score(gray, s, n_samples=max(8, XSEC_NSAMPLES//2), half=XSEC_HALF_PIX)
                     for s in pool], np.float64)
    cov  = np.array([line_board_coverage(s, mask_inner) for s in pool], np.float64)

    Lq0,Lq1 = np.quantile(L,0.10), np.quantile(L,0.90)
    len_norm = np.clip((L - Lq0) / (Lq1 - Lq0 + 1e-9), 0, 1)
    ang_norm = max(math.radians(ANG_NORM_DEG), 1e-3)
    dang_norm = np.clip(Adev/ang_norm, 0, 3.0)
    tq0,tq1 = np.quantile(tex,0.20), np.quantile(tex,0.90)
    tex_norm = np.clip((tex - tq0) / (tq1 - tq0 + 1e-9), 0, 1)
    Q = (1.0 - BETA_ANG*dang_norm) * (BETA_LEN0 + BETA_LEN1*len_norm) * (0.2 + TEXT_BONUS*tex_norm)

    # base cost
    C_base = np.zeros((len(pool), n), dtype=np.float64)
    for j, c in enumerate(centers):
        C_base[:, j] = W_RHO * np.abs(rho - c) + W_Q * (1.0 - Q)

    # hard gates (do once, reused for all separation levels)
    C = C_base.copy()
    far_gate = FAR_GATE_MULT * d_in
    out_of_band = (rho < lo - 0.25*d) | (rho > hi + 0.25*d)
    too_far     = np.min(np.abs(rho[:,None] - centers[None,:]), axis=1) > far_gate
    bad_cover   = cov < MIN_COVER
    kill = out_of_band | too_far | bad_cover
    if np.any(kill):
        C[kill, :] += 1e6  # make them unselectable

    def run_with_min_sep(min_sep_mult: float):
        """Run assignment + separation + refill with a given separation multiplier."""
        if len(pool) == 0:
            return pool

        # assignment
        rows, cols = linear_sum_assignment(C)
        sel_rows = rows[(cols >= 0) & (cols < n)]
        chosen = pool[sel_rows]

        # order by rho
        mids_ch = 0.5*(chosen[:,0:2] + chosen[:,2:4])
        rho_ch  = mids_ch @ nF
        order   = np.argsort(rho_ch)
        chosen  = chosen[order]
        rho_ch  = rho_ch[order]

        # separation
        min_sep = min_sep_mult * d_in
        ok = [True]*len(chosen)
        for i in range(1, len(chosen)):
            if abs(rho_ch[i] - rho_ch[i-1]) < min_sep:
                qi = Q[sel_rows[order[i]]]   * max(cov[sel_rows[order[i]]],   1e-9)
                qj = Q[sel_rows[order[i-1]]] * max(cov[sel_rows[order[i-1]]], 1e-9)
                ok[i-1 if qi >= qj else i] = False
        chosen = chosen[np.array(ok, bool)]
        used = set(sel_rows[order][np.array(ok, bool)])

        # refill if we dropped any
        if len(chosen) < n:
            best_center_cost = C.min(axis=1)
            for idx in np.argsort(best_center_cost):
                if idx in used or best_center_cost[idx] > 1e5:
                    continue
                s = pool[idx]
                r = float((0.5*(s[0:2]+s[2:4]) @ nF))
                if (r < lo) or (r > hi) or (cov[idx] < MIN_COVER):
                    continue
                if len(chosen) > 0 and min_sep > 0:
                    rho_cur = (0.5*(chosen[:,0:2]+chosen[:,2:4]) @ nF)
                    if np.min(np.abs(rho_cur - r)) < min_sep:
                        continue
                chosen = np.vstack([chosen, s]) if len(chosen) else np.array([s], np.float32)
                used.add(idx)
                if len(chosen) == n:
                    break

        # final in-band clamp
        if len(chosen) == 0:
            return chosen

        mids_fin = 0.5*(chosen[:,0:2] + chosen[:,2:4])
        rho_fin  = mids_fin @ nF
        keep = (rho_fin >= lo) & (rho_fin <= hi)
        chosen = chosen[keep]
        if len(chosen) == 0:
            return chosen

        # sort by rho again
        mids_fin = 0.5*(chosen[:,0:2] + chosen[:,2:4])
        rho_fin  = mids_fin @ nF
        chosen   = chosen[np.argsort(rho_fin)]
        return chosen

    # --- try with decreasing separation ---
    chosen_final = np.empty((0,4), dtype=np.float32)

    # e.g. MIN_SEP_MULT, MIN_SEP_MULT-0.1, ..., down to 0
    for step in range(8):
        cur_mult = max(MIN_SEP_MULT - 0.1*step, 0.0)
        chosen = run_with_min_sep(cur_mult)
        if len(chosen) == 0:
            continue
        if len(chosen) >= n:
            chosen_final = chosen
            break
        # keep best so far if we haven't reached n yet
        if len(chosen) > len(chosen_final):
            chosen_final = chosen

    if len(chosen_final) == 0:
        # totally failed; just return all pool sorted by rho
        mids_all = 0.5*(pool[:,0:2] + pool[:,2:4])
        rho_all  = mids_all @ nF
        pool_sorted = pool[np.argsort(rho_all)]
        # if still more than n, sub-sample
        if len(pool_sorted) > n:
            idx = np.linspace(0, len(pool_sorted)-1, n).astype(int)
            pool_sorted = pool_sorted[idx]
        return pool_sorted.astype(np.float32)

    # if we ended up with more than n, thin them out
    if len(chosen_final) > n:
        idx = np.linspace(0, len(chosen_final)-1, n).astype(int)
        chosen_final = chosen_final[idx]

    return chosen_final.astype(np.float32)


In [65]:
def detect_grid(board_img, out_dir):
    """
    Run LSD + clustering + Hungarian grid detection on a *cropped board image*.
    NO border contour detection; the whole image is treated as the board quad.
    """
    os.makedirs(out_dir, exist_ok=True)

    # optional: resize like before
    h0, w0 = board_img.shape[:2]
    if w0 > MAX_W:
        s = MAX_W / w0
        img = cv2.resize(board_img, (int(w0*s), int(h0*s)), cv2.INTER_AREA)
    else:
        img = board_img.copy()

    h, w  = img.shape[:2]
    diag  = math.hypot(w, h)

    # 1) edges
    edges = preprocess_edges(img)  # still uses your GAUSS_K, CANNY_*, etc.
    cv2.imwrite(str(out_dir / "01_edges.jpg"), edges)

    # 2) define "board quad" as the full image rectangle (skip contour detection)
    quad = np.array([
        [0,     0    ],
        [w - 1, 0    ],
        [w - 1, h - 1],
        [0,     h - 1]
    ], dtype=np.float32)

    # full mask = whole image is board
    mask_board = np.full((h, w), 255, np.uint8)
    mask_inner = erode_mask_by_ratio(mask_board, MASK_ERODE_RATIO_INNER)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3) LSD segments
    all_segs  = lsd_segments(gray, mask=mask_inner, min_len_ratio=0.0)
    long_segs = lsd_segments(gray, mask=mask_inner, min_len_ratio=MIN_LEN_RATIO)
    assert len(long_segs) >= 2, "need long segments for clustering"

    # 4) cluster long lines into two angle families
    fam1_long, fam2_long = cluster_two_families(long_segs)
    theta1 = mean_angle_of_family(fam1_long)
    theta2 = mean_angle_of_family(fam2_long)

    # enforce near-orthogonality fallback
    if angle_diff(theta1, theta2) < math.radians(25):
        theta2 = (theta1 + math.pi/2) % math.pi

    # 5) filter ALL segments by angle proximity
    ang_tol = math.radians(ANGLE_TOL_DEG)
    cand1, cand2 = [], []
    for s in all_segs:
        a = seg_angle(s)
        if angle_diff(a, theta1) <= ang_tol:
            cand1.append(s)
        elif angle_diff(a, theta2) <= ang_tol:
            cand2.append(s)
    cand1 = np.array(cand1, np.float32)
    cand2 = np.array(cand2, np.float32)

    # 6) merge + NMS + length filter
    fam1_merged = merge_family_preserve_angles(cand1, theta1, diag, RHO_EPS_RATIO, GAP_EPS_RATIO)
    fam2_merged = merge_family_preserve_angles(cand2, theta2, diag, RHO_EPS_RATIO, GAP_EPS_RATIO)
    fam1_merged = nms_by_rho(fam1_merged, theta1, diag, eps_ratio=0.010)
    fam2_merged = nms_by_rho(fam2_merged, theta2, diag, eps_ratio=0.010)

    def _filter_by_len(segs, min_len):
        if len(segs) == 0:
            return segs
        L = np.hypot(segs[:,2]-segs[:,0], segs[:,3]-segs[:,1])
        return segs[L >= min_len]

    min_len_after = POST_MIN_LEN_RATIO * diag
    fam1_before = _filter_by_len(fam1_merged, min_len_after)
    fam2_before = _filter_by_len(fam2_merged, min_len_after)

    # 7) drop rim-ish lines using inner band and coverage
    rmin1, rmax1, d1, nF1 = rho_span_and_d(theta1, quad, N_GRID)
    rmin2, rmax2, d2, nF2 = rho_span_and_d(theta2, quad, N_GRID)
    fam1_pool, d1 = filter_pool_by_inner_band(
        fam1_before, theta1, quad, N_GRID, INNER_BAND_MARGIN, nF1, mask_inner
    )
    fam2_pool, d2 = filter_pool_by_inner_band(
        fam2_before, theta2, quad, N_GRID, INNER_BAND_MARGIN, nF2, mask_inner
    )

    # debug: merged before select
    vis_before = draw_full_lines(img, fam1_pool, (255, 0, 0), 3)
    vis_before = draw_full_lines(vis_before, fam2_pool, (0, 255, 0), 3)
    # cv2.imwrite(str(out_dir / "03b_merged_before_select.jpg"), vis_before)

    # 8) strict selection of exactly N_GRID per family
    fam1 = select_n_hungarian_from_pool(fam1_pool, theta1, N_GRID, gray, quad, mask_inner)
    fam2 = select_n_hungarian_from_pool(fam2_pool, theta2, N_GRID, gray, quad, mask_inner)

    # 9) intersections
    dots = all_intersections(fam1, fam2, img.shape)

    # final visual
    vis_selected = draw_full_lines(img, fam1, (255,0,0), 3)
    vis_selected = draw_full_lines(vis_selected, fam2, (0,255,0), 3)
    cv2.imwrite(str(out_dir / "03c_merged_selected_NxN.jpg"), vis_selected)

    orig_vis = vis_selected.copy()
    for (x,y) in dots.astype(int):
        cv2.circle(orig_vis, (x,y), 3, (0,0,255), -1)
    # cv2.imwrite(str(out_dir / "04_output_on_original_NxN.jpg"), orig_vis)

    print("Saved debug to:", out_dir)
    print(f"Before select - Fam1: {len(fam1_before)} (pool {len(fam1_pool)}), "
          f"Fam2: {len(fam2_before)} (pool {len(fam2_pool)})")
    print(f"Selected - Fam1: {len(fam1)}, Fam2: {len(fam2)}, Intersections: {len(dots)}")

    return img, fam1, fam2, dots


In [66]:
def draw_hough_lines(img, vert_lines, horz_lines, out_path=None):
    """
    Draw vertical lines in blue, horizontal lines in green.
    """
    vis = img.copy()
    h, w = img.shape[:2]

    def draw_line(rho, theta, color):
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        # draw long line through the image
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(vis, (x1, y1), (x2, y2), color, 2, cv2.LINE_AA)

    for rho, theta in vert_lines:
        draw_line(rho, theta, (255, 0, 0))   # blue vertical

    for rho, theta in horz_lines:
        draw_line(rho, theta, (0, 255, 0))   # green horizontal

    if out_path is not None:
        cv2.imwrite(out_path, vis)

    return vis

In [67]:
# Debug

# IMG_PATH = "/kaggle/working/2_Move_student/03_frame420_START_clean.jpg"
# OUT = Path("/kaggle/working/vanish9x9_from_temp")

# # 1) read original frame
# orig = cv2.imread(IMG_PATH)
# assert orig is not None, f"not found: {IMG_PATH}"

# # 2) temp crop (YOLO board crop later)
# board_img = crop_board(orig)

# # 3) run “old style” grid detection on this board image
# img_proc, fam1, fam2, dots = detect_grid(board_img, OUT)

## Chess Piece Detecting

In [68]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("chess-lpnmj").project("chess-piece-detection-7alyg")
chess_detection_model = project.version(9).model 

loading Roboflow workspace...
loading Roboflow project...


In [69]:
def get_predictions_json(
    img_path: str, 
    model: Any, 
    confidence: int = 40, 
    overlap: int = 30
) -> Dict[str, Any]:
    """
    Takes an image path and a prediction model, and returns the object 
    detection results as a JSON dictionary.

    Args:
        img_path (str): The file path to the image to process.
        model (Any): The initialized prediction model object (e.g., Roboflow model).
        confidence (int): The confidence threshold for predictions (0-100). Default is 40.
        overlap (int): The NMS/overlap threshold (0-100). Default is 30.

    Returns:
        Dict[str, Any]: The prediction results in JSON format.
        
    Raises:
        FileNotFoundError: If the image path does not exist.
    """
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Error: Image file not found at path: {img_path}")
        
    # The core prediction logic from your original snippet
    pred_json = model.predict(img_path, confidence=confidence, overlap=overlap).json()
    
    return pred_json

In [70]:
# IMG_PATH = "/kaggle/working/2_Move_student/01_frame15_START_clean.jpg"

# pred_json = get_predictions_json(IMG_PATH, chess_detection_model)

# img = cv2.imread(IMG_PATH)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# for det in pred_json["predictions"]:
#     x = det["x"]
#     y = det["y"]
#     w = det["width"]
#     h = det["height"]
#     label = det["class"]
#     conf = det["confidence"]

#     # Roboflow gives center x,y + width/height
#     x1 = int(x - w / 2)
#     y1 = int(y - h / 2)
#     x2 = int(x + w / 2)
#     y2 = int(y + h / 2)

#     # draw rectangle
#     cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

#     # draw label
#     text = f"{label} {conf:.2f}"
#     cv2.putText(
#         img,
#         text,
#         (x1, max(y1 - 5, 0)),
#         cv2.FONT_HERSHEY_SIMPLEX,
#         0.4,
#         (255, 0, 0),
#         1,
#         cv2.LINE_AA
#     )

# # Show result
# plt.figure(figsize=(8, 8))
# plt.imshow(img)
# plt.axis("off")
# plt.show()

In [71]:
# pred_json

## Build board

In [72]:
from sklearn.cluster import KMeans

def dots_to_grid(dots, n_lines=9):
    """
    Convert unordered intersections into a [n_lines, n_lines, 2] grid of points
    sorted by image coordinates (row=top→bottom, col=left→right).
    dots: (N,2) float32 array from all_intersections
    """
    assert dots.shape[1] == 2
    ys = dots[:, 1].reshape(-1, 1)

    # Cluster by Y into n_lines horizontal groups
    kmeans = KMeans(n_clusters=n_lines, n_init=10, random_state=0)
    labels = kmeans.fit_predict(ys)
    centers = kmeans.cluster_centers_.ravel()

    # Map cluster index -> row index (0 = top)
    cluster_order = np.argsort(centers)
    cluster_to_row = {c: i for i, c in enumerate(cluster_order)}

    grid = np.zeros((n_lines, n_lines, 2), dtype=np.float32)

    for c in range(n_lines):
        # points in this cluster (row)
        pts = dots[labels == c]
        row_idx = cluster_to_row[c]

        # sort this row by x coordinate
        order_x = np.argsort(pts[:, 0])
        pts_sorted = pts[order_x]

        # if more than n_lines points (rare), keep closest n_lines
        if len(pts_sorted) > n_lines:
            pts_sorted = pts_sorted[:n_lines]

        # if fewer, pad by interpolation (very rare)
        if len(pts_sorted) < n_lines:
            # simple linear interpolation on x
            xs = np.linspace(pts_sorted[0,0], pts_sorted[-1,0], n_lines)
            ys_row = np.interp(xs, pts_sorted[:,0], pts_sorted[:,1])
            pts_sorted = np.stack([xs, ys_row], axis=1).astype(np.float32)

        grid[row_idx, :, :] = pts_sorted

    return grid  # shape (9,9,2)


In [73]:
CLASS_TO_FEN = {
    "white-pawn":   "P",
    "white-knight": "N",
    "white-bishop": "B",
    "white-rook":   "R",
    "white-queen":  "Q",
    "white-king":   "K",
    "black-pawn":   "p",
    "black-knight": "n",
    "black-bishop": "b",
    "black-rook":   "r",
    "black-queen":  "q",
    "black-king":   "k",
}

def build_board_from_predictions(pred_json, grid, min_area_thresh: float = 1.0):
    """
    Assign pieces to squares by maximizing bbox–square overlap area.

    pred_json: Roboflow-style prediction json (with x, y, width, height)
    grid: (9, 9, 2) intersection points in the same image coords

    Returns:
        board[8][8] with FEN chars or '.'
    """
    # start with '.' everywhere
    board = [["." for _ in range(8)] for _ in range(8)]

    # 1) Get vertical & horizontal "grid lines" from intersections
    #    (we'll treat squares as axis-aligned boxes between consecutive lines)
    row_lines_y = grid.mean(axis=1)[:, 1]  # (9,)
    col_lines_x = grid.mean(axis=0)[:, 0]  # (9,)

    # make sure they're sorted (usually already are, but just in case)
    row_lines_y = np.sort(row_lines_y)
    col_lines_x = np.sort(col_lines_x)

    # 2) Precompute square boxes: [xL, yT, xR, yB] for each (row, col)
    # row 0 = top, row 7 = bottom
    square_boxes = []
    for r in range(8):
        row_boxes = []
        y_top = row_lines_y[r]
        y_bot = row_lines_y[r + 1]
        for c in range(8):
            x_left  = col_lines_x[c]
            x_right = col_lines_x[c + 1]
            row_boxes.append((x_left, y_top, x_right, y_bot))
        square_boxes.append(row_boxes)

    def box_intersection_area(a, b):
        """
        a, b: (x1, y1, x2, y2)
        returns intersection area (>= 0)
        """
        ax1, ay1, ax2, ay2 = a
        bx1, by1, bx2, by2 = b
        ix1 = max(ax1, bx1)
        iy1 = max(ay1, by1)
        ix2 = min(ax2, bx2)
        iy2 = min(ay2, by2)
        if ix2 <= ix1 or iy2 <= iy1:
            return 0.0
        return float((ix2 - ix1) * (iy2 - iy1))

    # 3) Go through each detection, assign it to the square with max overlap
    for det in pred_json.get("predictions", []):
        cx = float(det["x"])
        cy = float(det["y"])
        w  = float(det["width"])
        h  = float(det["height"])
        cls_name = det["class"]
        conf = float(det.get("confidence", 0.0))

        fen_char = CLASS_TO_FEN.get(cls_name)
        if fen_char is None:
            continue

        # piece bbox (center x,y from Roboflow)
        x1 = cx - w / 2.0
        y1 = cy - h / 2.0
        x2 = cx + w / 2.0
        y2 = cy + h / 2.0
        piece_box = (x1, y1, x2, y2)

        # find square with max intersection area
        best_area = 0.0
        best_rc = None

        for r in range(8):
            for c in range(8):
                cell_box = square_boxes[r][c]
                area = box_intersection_area(piece_box, cell_box)
                if area > best_area:
                    best_area = area
                    best_rc = (r, c)

        # skip if no overlap (piece clearly outside grid)
        if best_rc is None or best_area < min_area_thresh:
            continue

        r, c = best_rc

        # handle conflicts: keep higher-confidence piece
        existing = board[r][c]
        if existing == ".":
            board[r][c] = (fen_char, conf)
        else:
            old_fen, old_conf = existing
            if conf > old_conf:
                board[r][c] = (fen_char, conf)

    # 4) unwrap tuples, leave '.' as empty
    for r in range(8):
        for c in range(8):
            if isinstance(board[r][c], tuple):
                board[r][c] = board[r][c][0]

    return board


## Pipeline

In [74]:
# Temp

def is_white_piece(p):
    return p.isalpha() and p.isupper()

def is_black_piece(p):
    return p.isalpha() and p.islower()

def rotate_board_for_white_bottom(board):
    """
    board: 8x8 list of lists with piece chars or '.'.
    Returns: board (maybe rotated 180°) so that white pieces
             are mostly on the bottom 4 ranks.
    """
    # count white pieces in top half (ranks 8–5) and bottom half (ranks 4–1)
    white_top = 0
    white_bot = 0
    for r in range(8):
        for c in range(8):
            p = board[r][c]
            if is_white_piece(p):
                if r < 4:
                    white_top += 1
                else:
                    white_bot += 1

    # if more white pieces are on top, rotate 180°
    if white_top > white_bot:
        rotated = [list(reversed(row)) for row in reversed(board)]
        return rotated
    else:
        return board


In [75]:
# debug function
def debug_overlay_grid_and_pieces(
    img_bgr: np.ndarray,
    grid: np.ndarray,
    pred_json: dict,
    out_path: Path = None,
    show: bool = True
):
    """
    Draw grid (from 9x9 intersections) + piece detections (from pred_json)
    on top of img_bgr for debugging.

    Args:
        img_bgr: original or processed board image (BGR, same coords as grid & preds)
        grid: (9,9,2) numpy array of intersection points (x,y)
        pred_json: Roboflow-style prediction JSON (with 'predictions' list)
        out_path: optional Path to save the debug image
        show: if True, display via matplotlib
    """
    vis = img_bgr.copy()

    # --- 1) Draw grid intersections as small circles ---
    for r in range(grid.shape[0]):
        for c in range(grid.shape[1]):
            x, y = grid[r, c]
            cv2.circle(vis, (int(x), int(y)), 3, (0, 255, 255), -1)  # yellow dots

    # --- 2) Draw grid lines (9 horizontal, 9 vertical) ---
    # horizontal lines
    for r in range(grid.shape[0]):
        pts = grid[r, :, :].astype(np.int32)
        cv2.polylines(vis, [pts], isClosed=False, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    # vertical lines
    for c in range(grid.shape[1]):
        pts = grid[:, c, :].astype(np.int32)
        cv2.polylines(vis, [pts], isClosed=False, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

    # --- 3) Draw piece detections (Roboflow uses center-x, center-y, width, height) ---
    for det in pred_json.get("predictions", []):
        cx = det["x"]
        cy = det["y"]
        w  = det["width"]
        h  = det["height"]
        cls_name = det.get("class", "?")
        conf = det.get("confidence", 0.0)

        # convert center + size → box corners
        x1 = int(cx - w / 2)
        y1 = int(cy - h / 2)
        x2 = int(cx + w / 2)
        y2 = int(cy + h / 2)

        # rectangle
        cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # label text
        label = f"{cls_name} {conf:.2f}"
        # put text slightly above box
        cv2.putText(
            vis,
            label,
            (x1, max(0, y1 - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 255),
            1,
            cv2.LINE_AA
        )

    # --- 4) Save if requested ---
    if out_path is not None:
        out_path = Path(out_path)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), vis)

    # --- 5) Show if requested ---
    if show:
        plt.figure(figsize=(6, 10))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Grid + Piece Predictions Overlay")
        plt.show()

    return vis

In [76]:
def frame_to_board(
    frame_bgr: np.ndarray,
    grid_out_dir: Path,
    piece_model,                    
    piece_confidence: int = 40,
    piece_overlap: int = 30
):
    """
    Full pipeline:
      BGR frame -> crop_board -> rotate (white at bottom) -> grid detection -> piece detection.
    """
    # 1) Crop board (currently identity; replace later with YOLO board crop)
    board_img = crop_board(frame_bgr)

    # 2) Rotate so that white side is at the bottom
    board_img = rotate(board_img)

    # 3) Detect grid on the rotated board image
    img_proc, fam1, fam2, dots = detect_grid(board_img, grid_out_dir)

    # 4) Convert intersections to 9x9 grid points
    grid = dots_to_grid(dots, n_lines=N_GRID)  # N_GRID = 9

    # 5) Run piece detection on the same processed image
    tmp_path = str(grid_out_dir / "board_for_pieces.jpg")
    cv2.imwrite(tmp_path, img_proc)

    pred_json = get_predictions_json(
        tmp_path,
        model=piece_model,
        confidence=piece_confidence,
        overlap=piece_overlap
    )
    
    # 6) Build board matrix (now consistent with "white at bottom")
    board = build_board_from_predictions(pred_json, grid)

    # Temp
    rotate_board = rotate_board_for_white_bottom(board)

    # Debug
    # debug_path = "debug_overlay.jpg"

    # debug_overlay_grid_and_pieces(
    #     img_bgr=img_proc,  
    #     grid=grid,
    #     pred_json=pred_json,
    #     out_path=debug_path,
    #     show=True
    # )
    
    return rotate_board, grid, pred_json


In [77]:
from pathlib import Path

# 1) pick a clean frame generated earlier
IMG_PATH = "/kaggle/working/2_Move_student/01_frame15_START_clean.jpg"
OUT_DIR  = Path("/kaggle/working/fen_pipeline_01")
OUT_DIR.mkdir(parents=True, exist_ok=True)

frame = cv2.imread(IMG_PATH)
assert frame is not None, f"not found: {IMG_PATH}"

board, grid, pred_json = frame_to_board(
    frame_bgr=frame,
    grid_out_dir=OUT_DIR,
    piece_model=chess_detection_model,    
    piece_confidence=40,
    piece_overlap=30
)

for r in board:
    print(r)

Saved debug to: /kaggle/working/fen_pipeline_01
Before select - Fam1: 12 (pool 12), Fam2: 12 (pool 12)
Selected - Fam1: 9, Fam2: 9, Intersections: 81
['r', '.', '.', 'q', 'k', 'b', 'n', 'N']
['p', 'p', 'p', 'b', '.', '.', 'p', '.']
['.', '.', '.', 'b', '.', '.', '.', '.']
['n', '.', '.', 'P', 'p', '.', '.', '.']
['.', 'P', '.', '.', 'P', '.', '.', '.']
['.', '.', 'N', '.', '.', '.', '.', '.']
['P', '.', 'P', '.', '.', '.', 'P', 'P']
['R', '.', '.', 'Q', '.', 'B', '.', 'R']


In [78]:
# 1) pick a clean frame generated earlier
IMG_PATH = "/kaggle/working/2_Move_student/03_frame420_START_clean.jpg"
OUT_DIR  = Path("/kaggle/working/fen_pipeline_01")
OUT_DIR.mkdir(parents=True, exist_ok=True)

frame = cv2.imread(IMG_PATH)
assert frame is not None, f"not found: {IMG_PATH}"

board, grid, pred_json = frame_to_board(
    frame_bgr=frame,
    grid_out_dir=OUT_DIR,
    piece_model=chess_detection_model,    
    piece_confidence=40,
    piece_overlap=30
)

for r in board:
    print(r)

Saved debug to: /kaggle/working/fen_pipeline_01
Before select - Fam1: 12 (pool 12), Fam2: 12 (pool 12)
Selected - Fam1: 9, Fam2: 9, Intersections: 81
['r', '.', '.', '.', 'k', 'b', 'n', 'N']
['p', 'p', 'p', 'b', '.', '.', 'p', '.']
['.', '.', '.', 'p', '.', '.', '.', '.']
['n', '.', '.', 'P', 'p', '.', '.', '.']
['.', 'P', '.', '.', 'P', '.', '.', 'q']
['.', '.', 'N', '.', '.', '.', '.', '.']
['P', '.', 'P', '.', '.', '.', 'P', 'P']
['R', '.', '.', 'Q', '.', 'B', '.', 'R']


In [79]:
# 1) pick a clean frame generated earlier
IMG_PATH = "/kaggle/working/2_Move_student/05_frame1230_START_clean.jpg"
OUT_DIR  = Path("/kaggle/working/fen_pipeline_01")
OUT_DIR.mkdir(parents=True, exist_ok=True)

frame = cv2.imread(IMG_PATH)
assert frame is not None, f"not found: {IMG_PATH}"

board, grid, pred_json = frame_to_board(
    frame_bgr=frame,
    grid_out_dir=OUT_DIR,
    piece_model=chess_detection_model,    
    piece_confidence=40,
    piece_overlap=30
)

for r in board:
    print(r)

Saved debug to: /kaggle/working/fen_pipeline_01
Before select - Fam1: 11 (pool 11), Fam2: 12 (pool 12)
Selected - Fam1: 9, Fam2: 9, Intersections: 81
['.', 'r', '.', '.', '.', 'k', 'b', 'N']
['.', 'p', 'p', 'p', 'b', '.', '.', 'p']
['.', '.', '.', '.', 'p', '.', '.', '.']
['.', 'n', '.', '.', 'P', 'p', '.', '.']
['.', '.', 'P', '.', '.', 'P', '.', 'q']
['.', '.', '.', 'N', '.', '.', '.', 'P']
['.', 'P', '.', 'P', '.', '.', '.', 'P']
['.', 'R', '.', '.', 'Q', '.', 'B', 'R']


## PGN

In [ ]:
!pip install chess

In [ ]:
import chess
import chess.pgn

In [ ]:
def list_to_fen(board_list, turn='w', castling='KQkq', ep='-', half='0', full='1'):
    fen_rows = []
    for row in board_list:
        empty_count = 0
        current_row_string = ""
        for square in row:
            if square == '.':
                empty_count += 1
            else:
                if empty_count > 0:
                    current_row_string += str(empty_count)
                    empty_count = 0
                current_row_string += square
        if empty_count > 0:
            current_row_string += str(empty_count)
        fen_rows.append(current_row_string)
    piece_placement = "/".join(fen_rows)
    return f"{piece_placement} {turn} {castling} {ep} {half} {full}"

In [ ]:
def get_castling_rights(board_list):
    castling = ""
    
    # --- WHITE CHECK (Row 7 in your list) ---
    # King must be at e1 (Row 7, Col 4)
    if board_list[7][4] == 'K':
        # Check King-side Rook at h1 (Row 7, Col 7)
        if board_list[7][7] == 'R':
            castling += "K"
        # Check Queen-side Rook at a1 (Row 7, Col 0)
        if board_list[7][0] == 'R':
            castling += "Q"
            
    # --- BLACK CHECK (Row 0 in your list) ---
    # King must be at e8 (Row 0, Col 4)
    if board_list[0][4] == 'k':
        # Check King-side Rook at h8 (Row 0, Col 7)
        if board_list[0][7] == 'r':
            castling += "k"
        # Check Queen-side Rook at a8 (Row 0, Col 0)
        if board_list[0][0] == 'r':
            castling += "q"
            
    # If no one is on home squares, return "-"
    if castling == "":
        return "-"
        
    return castling

In [ ]:
def detect_active_color(current_frame, next_frame):
    
    # We scan the board to find the piece that LEFT its square
    for r in range(8):
        for c in range(8):
            old_val = current_frame[r][c]
            new_val = next_frame[r][c]
            
            if old_val != '.' and new_val == '.':
                if old_val.isupper():
                    return 'w' 
                else:
                    return 'b' 
    return 'w'

In [ ]:
def boards_to_pgn_from_matrices(board_list, start_move_num=1):
    # STEP 1: Setup the FIRST state specifically
    # We need to construct the very first FEN manually so the game knows where to start.
    first_fen = list_to_fen(
        board_list[0],  # The image list
        turn=detect_active_color(board_list[0], board_list[1]),
        full=str(start_move_num),
        castling=get_castling_rights(board_list[0])
    )
    
    # Create the root game object
    game = chess.pgn.Game()
    
    # Create a board with our custom starting position
    board = chess.Board(first_fen)
    
    game.setup(board) 
    
    node = game

    for i in range(len(board_list) - 1):
        next_fen_string = list_to_fen(board_list[i+1]) # Only needs piece placement
        
        # Determine who is moving based on the board state
        # The board object tracks whose turn it is automatically
        
        move_found = None
        for move in board.legal_moves:
            board.push(move)
            
            # Compare only the piece positions (split()[0])
            if board.fen().split()[0] == next_fen_string.split()[0]:
                move_found = move
                board.pop()
                break
            board.pop()
            
        if move_found:
            board.push(move_found)
            node = node.add_variation(move_found)
        else:
            print(f"Error at frame {i}")
    
    exporter = chess.pgn.StringExporter(headers=False, variations=True, comments=True)
    move_text = game.accept(exporter)
    
    return move_text

## Final Pipeline

In [ ]:
def video_to_boards_and_pgn(
    video_path: str,
    hand_split_dir: str,
    work_root: Path,
    piece_model,
    piece_confidence: int = 40,
    piece_overlap: int = 30,
):
    """
    Full pipeline:
      1) From video_path, run save_clean_chess_frames -> JPEG frames.
      2) For each stable 'clean' START frame, run frame_to_board().
      3) Collect board matrices.
      4) Build a temp PGN from the board sequence.

    Returns:
      board_list: [board0, board1, ...] each 8x8 matrix
      pgn_str:    PGN string (temp reconstruction)
    """

    hand_split_dir = Path(hand_split_dir)
    work_root = Path(work_root)
    hand_split_dir.mkdir(parents=True, exist_ok=True)
    work_root.mkdir(parents=True, exist_ok=True)

    # --- 1) Split video into clean/blocked chunks using your YOLO hand model ---
    save_clean_chess_frames(str(video_path), output_folder=str(hand_split_dir), save_step=True)

    # --- 2) Collect the frames that correspond to stable clean states ---
    all_jpgs = sorted(hand_split_dir.glob("*.jpg"))

    clean_start_frames = [
        p for p in all_jpgs
        if "START_clean" in p.name
    ]

    if not clean_start_frames:
        print("No START_clean frames found; check video or hand detector.")
        return [], ""

    board_list = []

    # --- 3) For each 'clean' frame, run your full board pipeline ---
    for idx, frame_path in enumerate(clean_start_frames):
        frame_bgr = cv2.imread(str(frame_path))
        if frame_bgr is None:
            print(f"Warning: could not read frame {frame_path}")
            continue

        grid_out_dir = work_root / f"pos_{idx:02d}"
        grid_out_dir.mkdir(parents=True, exist_ok=True)

        try:
            board, grid, pred_json = frame_to_board(
                frame_bgr=frame_bgr,
                grid_out_dir=grid_out_dir,
                piece_model=piece_model,
                piece_confidence=piece_confidence,
                piece_overlap=piece_overlap,
            )
            board_list.append(board)

        except AssertionError as e:
            print(f"[Frame {idx}] Grid detection failed: {e}")
        except Exception as e:
            print(f"[Frame {idx}] Unexpected error: {e}")

    if not board_list:
        print("No boards successfully reconstructed.")
        return [], ""
        
    # --- 4) Convert board sequence to PGN ---
    pgn_str = boards_to_pgn_from_matrices(board_list, start_move_num=1)

    return board_list, pgn_str

In [87]:
video_path     = "/kaggle/input/cu-chess-detection-2025/Chess Detection Competition/test_videos/2_Move_rotate_student.mp4"
hand_split_dir = "/kaggle/working/clean_frames"
work_root      = Path("/kaggle/working/fen_pipeline_validation")

boards, pgn = video_to_boards_and_pgn(
    video_path=video_path,
    hand_split_dir=hand_split_dir,
    work_root=work_root,
    piece_model=chess_detection_model,       
    piece_confidence=40,
    piece_overlap=30,
)

print("Number of board states:", len(boards))
print("Temp PGN:\n")
print(pgn)

Processing... Saving ordered chunks to '//kaggle/working/clean_frames'
[001] Started clean at frame 15
[002] Switched to blocked at frame 75
[003] Switched to clean at frame 420
[004] Switched to blocked at frame 1155
[005] Switched to clean at frame 1230
Done.
Saved debug to: /kaggle/working/fen_pipeline_validation/pos_00
Before select - Fam1: 12 (pool 12), Fam2: 14 (pool 13)
Selected - Fam1: 9, Fam2: 9, Intersections: 81
Saved debug to: /kaggle/working/fen_pipeline_validation/pos_01
Before select - Fam1: 13 (pool 12), Fam2: 12 (pool 12)
Selected - Fam1: 9, Fam2: 9, Intersections: 81
Saved debug to: /kaggle/working/fen_pipeline_validation/pos_02
Before select - Fam1: 13 (pool 12), Fam2: 13 (pool 13)
Selected - Fam1: 9, Fam2: 9, Intersections: 81
Number of board states: 3
Temp PGN:




In [88]:
print(boards)

[[['.', '.', '.', '.', '.', '.', '.', '.'], ['.', 'R', '.', '.', 'n', 'p', 'r', '.'], ['.', '.', '.', 'P', '.', 'p', '.', '.'], ['.', 'P', 'N', '.', '.', 'p', '.', '.'], ['.', 'Q', '.', '.', 'P', 'p', 'q', '.'], ['.', 'B', '.', 'P', 'p', '.', 'b', '.'], ['.', 'P', '.', '.', '.', 'p', 'n', '.'], ['.', 'R', '.', '.', '.', '.', 'R', '.']], [['.', '.', '.', '.', '.', '.', '.', '.'], ['.', 'R', '.', '.', 'n', '.', 'p', '.'], ['.', '.', '.', 'P', '.', '.', 'p', '.'], ['Q', 'P', 'N', '.', 'P', 'p', 'p', '.'], ['.', '.', '.', 'P', 'p', '.', 'k', '.'], ['.', 'B', '.', '.', '.', '.', 'b', '.'], ['.', 'P', '.', '.', '.', '.', 'p', '.'], ['R', 'P', '.', 'q', '.', '.', 'R', '.']], [['.', 'R', '.', '.', 'q', '.', 'R', '.'], ['.', 'n', 'p', '.', '.', 'P', '.', '.'], ['.', 'b', '.', '.', '.', '.', 'B', '.'], ['.', 'k', '.', 'p', 'P', '.', '.', '.'], ['.', '.', 'b', 'P', '.', '.', 'Q', '.'], ['.', '.', 'p', '.', '.', 'N', 'P', '.'], ['.', '.', 'p', '.', 'P', '.', '.', '.'], ['.', 'r', 'p', 'n', '.', '.